# Exciton Decompositions Analysis

There are qualititative and quantative analysis, for qualititive you can use summarize eigenvectors, just know the largest band to band transition and largest k point, then you will learn the band composition into parts.

For ploteigenvectors.py is also a qualititive analysis, summarize eigenvectors is also semi-qualiative, because of limited information: 

It only tells you about wtot, wmax and ikmax for different band to band transitions for each Exciton S - You cannot resolve K-point from this information, thus cannot safely project to atomic orbitals.

`wtot = sum_k |A_vck|^2. wmax = max_k |A_vck|^2. |A_vc (ikmax)|^2 = wmax.`


**To do a proper quantitative approach:**

Read eigenvectors.h5 to extract K resolved decomposition into different band to band transition, in combination with pdos information, I can calculate the percentage for perovskite/non-perovskite of electrons and holes for the every excitons, similar information are also calculated when I did the ECF-noshift plot, where I can see the localization of electrons and holes for each exciton in real space.

In [17]:
import pandas as pd
import numpy as np

## Step 0: Find the K list correspondance from bandstructure.dat (unfolded) with the K list from kgrid.log (folded)

### 0.1 Process K list from Kgrid.log (folded)

In [43]:
## Process the kgrid.log, find the correpondance with uniform grid and irrduciible grid used to calculate WFN_fi

def process_data(file_content):
    lines = file_content.split('\n')

    for i, line in enumerate(lines):
        if "k-points in the original uniform grid" in line:
            # Next line contains the number of rows to read
            num_rows = int(lines[i + 1].strip())
            # Data starts from the next line
            start_line = i + 2
            break

    data = []
    for line in lines[start_line:start_line + num_rows]:
        # Split the line into columns and convert to appropriate types
        columns = line.split()
        row = [int(columns[0])] + [float(c) for c in columns[1:5]] + [int(columns[5]), columns[6]]
        data.append(row)
    unfolded_idx = {}
    cnt = 0
    processed_data = []

    for row in data:
        row_number = row[0]
        reference_row = row[5]

        # If the sixth column is 0, increment the count
        if reference_row == 0:
            cnt = cnt + 1
            unfolded_idx[row_number] = cnt
        else:
            # If the sixth column is not 0, set the count to the count of the referenced row
            unfolded_idx[row_number] = unfolded_idx[reference_row]
        processed_data.append(row + [unfolded_idx[row_number]])

    return processed_data

process_data(open('kgrid.log').read())

[[1, 0.0, 0.0, 0.0, 1.0, 0, '---', 1],
 [2, 0.0, 0.0, 0.083333333, 2.0, 0, '---', 2],
 [3, 0.0, 0.0, 0.166666667, 2.0, 0, '---', 3],
 [4, 0.0, 0.0, 0.25, 2.0, 0, '---', 4],
 [5, 0.0, 0.0, 0.333333333, 2.0, 0, '---', 5],
 [6, 0.0, 0.0, 0.416666667, 2.0, 0, '---', 6],
 [7, 0.0, 0.0, 0.5, 1.0, 0, '---', 7],
 [8, 0.0, 0.0, 0.583333333, 0.0, 6, 'r03', 6],
 [9, 0.0, 0.0, 0.666666667, 0.0, 5, 'r03', 5],
 [10, 0.0, 0.0, 0.75, 0.0, 4, 'r03', 4],
 [11, 0.0, 0.0, 0.833333333, 0.0, 3, 'r03', 3],
 [12, 0.0, 0.0, 0.916666667, 0.0, 2, 'r03', 2],
 [13, 0.0, 0.083333333, 0.0, 2.0, 0, '---', 8],
 [14, 0.0, 0.083333333, 0.083333333, 4.0, 0, '---', 9],
 [15, 0.0, 0.083333333, 0.166666667, 4.0, 0, '---', 10],
 [16, 0.0, 0.083333333, 0.25, 4.0, 0, '---', 11],
 [17, 0.0, 0.083333333, 0.333333333, 4.0, 0, '---', 12],
 [18, 0.0, 0.083333333, 0.416666667, 4.0, 0, '---', 13],
 [19, 0.0, 0.083333333, 0.5, 2.0, 0, '---', 14],
 [20, 0.0, 0.083333333, 0.583333333, 0.0, 18, 'r03', 13],
 [21, 0.0, 0.083333333, 0.66666

# Process bandstructure.dat (unfolded)

## Step 1: Decomposition of excitons into the contribution of band-to-band transitions at different k points

Specifically, I read and store sum_v/c |Acvk|^2 for each (S,k) pair for each band in c, v respectively. Then I can also plot the distribution of the contribution of each band-to-band transition to the exciton wavefunction as I did using the script plot_eigenvectors_kloop.py.

file:///Users/yinanchen/PhD/2022_Intergrowth_Hema/APD1_Na/absorption/k104/evec_component_k=0.png
file:///Users/yinanchen/PhD/2022_Intergrowth_Hema/APD1_Na/absorption/k104/evec_component_k=-1.png

In [18]:
with open('cl.projbands', 'r') as f:
    lines = f.readlines()

comments = []
for line in lines:
    if line.startswith('#'):
        comments.append(line)
    else:
        break

data = []
for line in lines:
    if not line.startswith('#'):
        data.append(line)


In [19]:
# convert the list of strings to a list of lists, 2D array
data = [line.split() for line in data]

In [20]:
# conver the 2d array to a dataframe, and convert the strings to floats
df = pd.DataFrame(data).astype(float)

In [21]:
df

,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
0,1.0,0.00000,-21.19916,0.968,0.0,0.0,0.0,0.0,0.0,0.0,...,0.003,0.003,0.018,0.018,0.003,0.002,0.002,0.003,0.003,0.003
1,1.0,0.00000,-21.19916,0.968,0.0,0.0,0.0,0.0,0.0,0.0,...,0.003,0.003,0.018,0.018,0.003,0.002,0.002,0.003,0.003,0.003
2,1.0,0.00000,-21.19884,0.968,0.0,0.0,0.0,0.0,0.0,0.0,...,0.003,0.003,0.018,0.018,0.003,0.002,0.002,0.003,0.003,0.003
3,1.0,0.00000,-21.19884,0.968,0.0,0.0,0.0,0.0,0.0,0.0,...,0.003,0.003,0.018,0.018,0.003,0.002,0.002,0.003,0.003,0.003
4,1.0,0.00000,-21.19682,0.976,0.0,0.0,0.0,0.0,0.0,0.0,...,0.003,0.003,0.018,0.018,0.003,0.002,0.002,0.003,0.003,0.003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61875,182.0,60.43921,6.17376,0.308,0.0,0.0,0.0,0.0,0.0,0.0,...,0.002,0.002,0.005,0.005,0.002,0.003,0.003,0.002,0.002,0.002
61876,182.0,60.43921,6.25687,0.496,0.0,0.0,0.0,0.0,0.0,0.0,...,0.005,0.005,0.003,0.003,0.004,0.000,0.000,0.004,0.005,0.005
61877,182.0,60.43921,6.25687,0.496,0.0,0.0,0.0,0.0,0.0,0.0,...,0.005,0.005,0.003,0.003,0.004,0.000,0.000,0.004,0.005,0.005
61878,182.0,60.43921,6.26715,0.528,0.0,0.0,0.0,0.0,0.0,0.0,...,0.002,0.002,0.003,0.003,0.004,0.003,0.003,0.004,0.002,0.002


In [22]:
import numpy as np
import h5py as h5
import matplotlib.pyplot as plt
contrib_c_dict = {}
contrib_v_dict = {}

def plot_for_k(k_select):
    f = h5.File('eigenvectors.h5','r')
    evals = f['exciton_data/eigenvalues'][()]
    evc = f['exciton_data/eigenvectors'][()]
    evc = evc[0,:,:,:,:,0,:]
    (nS, nk, nc, nv, _) = evc.shape

    if k_select == -1:
        pass
    else:
        evc = evc[:,[k_select],:,:,:]

    for iN_S in range(nS):
        if e_min<evals[iN_S]<e_high:
            temp_contrib_cv = np.sum(abs(evc[iN_S,:,:,:,0]+evc[iN_S,:,:,:,1]*1j)**2, axis=0) # Summarize_eigenvectors and Ploteigenvectors - For a certain k and c, calculating the sum of Acvk across all vs, and vice versa, this is the correct way to do it, because for a certain k, valence band pdos idoes not change when we fix a v, and vice versa, But you ignore the correlation between c and v, thus you miss the ECF, but you can only plot ECF_noshift
            temp_contrib_v = np.sum(temp_contrib_cv, axis=0)
            temp_contrib_c = np.sum(temp_contrib_cv, axis=1)

            contrib_c_dict[(iN_S,k_select)] = temp_contrib_c
            contrib_v_dict[(iN_S,k_select)] = temp_contrib_v
    return nS, nk, nc, nv

In [23]:
#nk = 400 # ! Hard-code the number of k-points for now
nk = 576
e_min = 0
e_high = 5

for k in range(0,nk):
    nS, nk, nc, nv = plot_for_k(k)
    #print(contrib_c_dict)
    #print(contrib_v_dict)
print(nk)

576


In [24]:
contrib_c_dict

{(0,
  0): array([7.99165928e-02, 7.27211265e-02, 2.27365265e-09, 2.06923364e-09,
        7.48807598e-11, 5.84837746e-11, 9.34584453e-10, 1.55367623e-09,
        4.86255548e-11, 8.57541967e-11, 4.93003428e-09, 4.25342600e-09,
        2.04556853e-10, 1.43229819e-10, 2.32839593e-10, 1.72077228e-10,
        1.96999472e-11, 1.95100115e-11, 3.14501848e-12, 4.13224008e-12]),
 (1,
  0): array([7.13168427e-02, 8.13290851e-02, 1.87702347e-09, 2.04665518e-09,
        5.80858588e-11, 8.36731286e-11, 1.55782207e-09, 9.35397238e-10,
        8.71625837e-11, 4.89506034e-11, 4.25103133e-09, 5.11965671e-09,
        1.90340970e-10, 2.51621115e-10, 1.74403928e-10, 2.39790318e-10,
        1.46610848e-11, 1.51348695e-11, 3.52779590e-12, 2.44460403e-12]),
 (2,
  0): array([7.96256380e-02, 7.30367334e-02, 1.66140016e-09, 1.29063489e-09,
        7.69153019e-11, 6.19865576e-11, 1.22719820e-09, 1.22283178e-09,
        7.54763586e-11, 7.74016038e-11, 4.73297490e-09, 4.27764976e-09,
        9.44466436e-11, 8.9547

In [25]:
nS, nk, nc, nv

(10, 576, 20, 20)

In [26]:
# find the corresponding pdos for each k from df
# Count the number of k points in df, should be consistent with nk
n_k = len(df[0].unique())
n_k

182

## Step 2: Calculate the kpdos on a WFN_fi calculation

Now that we have the decomposition of excitons into the contribution of band-to-band transitions for each (S,k) pair.

For every k, find the corresponding pdos from K-resolved Projected DOS on WFN_fi.

In [27]:
# Before that, we need to isolating only the middle nv+nc rows for each k, 
# corresponding to the nv valence and nc conduction bands

# First group df by the first column, then operate on each group
# For each group, take out the middle nv+nc rows around fermi, i.e. fermi - nv : fermi + nc
# ! Locate the position of fermi, i.e. the position where df[2] changes from negative to positive. 
#fermi = 408 # ! VBM - start + 1 , hard-coded for APD2_Li now, read from projbands file
fermi = 280 # ! for HOCN
conduction_df = df.groupby(0).apply(lambda x: x.iloc[fermi:fermi+nc,:])
valence_df = df.groupby(0).apply(lambda x: x.iloc[fermi-nv:fermi,:])
reversed_valence_df = valence_df.iloc[::-1,:]
reversed_valence_df

0         1        2      3    4    5    6    7    8    9    \
0                                                                            
182.0 61819  182.0  60.43921 -0.38161  0.976  0.0  0.0  0.0  0.0  0.0  0.0   
      61818  182.0  60.43921 -0.38161  0.976  0.0  0.0  0.0  0.0  0.0  0.0   
      61817  182.0  60.43921 -0.39134  0.968  0.0  0.0  0.0  0.0  0.0  0.0   
      61816  182.0  60.43921 -0.39134  0.968  0.0  0.0  0.0  0.0  0.0  0.0   
      61815  182.0  60.43921 -0.86214  0.952  0.0  0.0  0.0  0.0  0.0  0.0   
...            ...       ...      ...    ...  ...  ...  ...  ...  ...  ...   
1.0   264      1.0   0.00000 -1.03106  0.956  0.0  0.0  0.0  0.0  0.0  0.0   
      263      1.0   0.00000 -1.09655  0.964  0.0  0.0  0.0  0.0  0.0  0.0   
      262      1.0   0.00000 -1.09655  0.964  0.0  0.0  0.0  0.0  0.0  0.0   
      261      1.0   0.00000 -1.17449  0.968  0.0  0.0  0.0  0.0  0.0  0.0   
      260      1.0   0.00000 -1.17449  0.968  0.0  0.0  0.0  0.0  0.0  0.0   

             ...  374  375  376  377  378  379  380  381  382  383  
0            ...                                                    
182.0 61819  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
      61818  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
      61817  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
      61816  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
      61815  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
...          ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  
1.0   264    ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
      263    ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
      262    ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
      261    ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
      260    ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[3640 rows x 384 columns]

In [28]:
# This step is to check the vs are negative and cs are positive
# To ensure that the hard-coded fermi is correct

valence_df.loc[1]
print(valence_df.loc[1][2].values)
print(reversed_valence_df.loc[1][2].values)
print(conduction_df.loc[1][2].values)

[-1.17449e+00 -1.17449e+00 -1.09655e+00 -1.09655e+00 -1.03106e+00
 -1.03106e+00 -9.78940e-01 -9.78940e-01 -9.26220e-01 -9.26220e-01
 -8.89050e-01 -8.89050e-01 -8.28840e-01 -8.28840e-01 -8.23720e-01
 -8.23720e-01 -5.82930e-01 -5.82930e-01 -6.40000e-04 -6.40000e-04]
[-6.40000e-04 -6.40000e-04 -5.82930e-01 -5.82930e-01 -8.23720e-01
 -8.23720e-01 -8.28840e-01 -8.28840e-01 -8.89050e-01 -8.89050e-01
 -9.26220e-01 -9.26220e-01 -9.78940e-01 -9.78940e-01 -1.03106e+00
 -1.03106e+00 -1.09655e+00 -1.09655e+00 -1.17449e+00 -1.17449e+00]
[0.92845 0.92845 1.85164 1.85164 1.98649 1.98649 2.1691  2.1691  2.17434
 2.17434 2.61565 2.61565 3.12082 3.12082 3.37417 3.37417 3.3925  3.3925
 3.50952 3.50952]


In [29]:
# To correspond to the contribution of valence bands (counting from the VBM) and conduction bands (counting from the CBM)
# We need to reverse the order of the valence bands

reversed_valence_df.loc[40]
#conduction_df.loc[40]

,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
13539,40.0,12.14342,-0.36931,0.980,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13538,40.0,12.14342,-0.36932,0.980,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13537,40.0,12.14342,-0.44931,0.964,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13536,40.0,12.14342,-0.44933,0.964,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13535,40.0,12.14342,-0.85080,0.960,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13534,40.0,12.14342,-0.85084,0.960,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13533,40.0,12.14342,-0.86488,0.968,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13532,40.0,12.14342,-0.86496,0.968,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13531,40.0,12.14342,-0.91633,0.960,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13530,40.0,12.14342,-0.91641,0.960,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Combine the knowledge of K resolved pdos and exciton decomposition into bands

For each k point, find out the decomposition of bands into atomic orbitals

In [30]:
contrib_v_dict[(s_select,k_select)].sum()

NameError: name 's_select' is not defined

In [ ]:
# Now we use contrib_v_dict[(s,k)] as the weight to multiply with the pdos of the valence band at k = k_select
# Then we sum over all k points, and we get the pdos of the valence band

# ! Note, Question: should we normalize the weight? i.e. divided by the sum of the weights over all vs or cs for each k point?
# ! Answer: No, because each k point in BZ should have a different total weight as seen by the plot_eigenvectors.py plots,
# #! so we should not normalize the weight for each k point.

# ! Also, I am not normalizing pdos (not divided by TOTAL (ranging from 0.7 - 0.9) in column 4), 
# ! using their original pdos at (k, n), where n is the band index.

s_select = 9
result_df_valence = pd.DataFrame()
result_df_conduction = pd.DataFrame()
for k_select in range(0,n_k):
    weighted_sum_valence = reversed_valence_df.loc[k_select+1].mul(contrib_v_dict[(s_select,k_select)], axis=0).sum()
    # \ / contrib_v_dict[(s_select,k_select)].sum()
    result_df_weighted_sum_valence = pd.DataFrame([weighted_sum_valence])

    weighted_sum_conduction = conduction_df.loc[k_select+1].mul(contrib_c_dict[(s_select,k_select)], axis=0).sum()
    # \ / contrib_c_dict[(s_select,k_select)].sum()
    result_df_weighted_sum_conduction = pd.DataFrame([weighted_sum_conduction])

    # append all k points into a single dataframe
    #result_df = result_df.append(result_df_weighted_sum)
    result_df_valence = pd.concat([result_df_valence, result_df_weighted_sum_valence], axis=0)
    result_df_conduction = pd.concat([result_df_conduction, result_df_weighted_sum_conduction], axis=0)

# sum over all k points, row-wise
print(result_df_valence.sum(axis=0))
print(result_df_conduction.sum(axis=0))
result_df_conduction

0      64.716237
1       9.459972
2      -0.105060
3       0.918252
4       0.000000
         ...    
655     0.000000
656     0.000000
657     0.000000
658     0.000000
659     0.000000
Length: 660, dtype: float64
0      6.471624e+01
1      9.459972e+00
2      1.215197e+00
3      9.524720e-01
4      0.000000e+00
           ...     
655    1.035891e-09
656    1.035891e-09
657    0.000000e+00
658    1.437415e-10
659    1.415609e-10
Length: 660, dtype: float64


,0,1,2,3,4,5,6,7,8,9,...,650,651,652,653,654,655,656,657,658,659
0,0.000899,0.000000,0.001223,0.000852,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,6.556479e-12,6.556479e-12,0.0,1.260585e-12,1.260585e-12,0.0,0.0,0.0
0,0.005568,0.000237,0.003788,0.002639,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.738439e-12,5.738439e-12,0.0,1.171830e-12,1.171830e-12,0.0,0.0,0.0
0,0.006795,0.000386,0.003082,0.002147,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,8.048956e-12,8.048956e-12,0.0,1.237815e-12,1.237815e-12,0.0,0.0,0.0
0,0.023744,0.001517,0.008077,0.005627,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.442767e-10,1.442767e-10,0.0,7.961720e-13,7.961720e-13,0.0,0.0,0.0
0,0.000216,0.000023,0.000061,0.000041,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,6.441853e-12,6.441853e-12,0.0,1.302182e-12,1.302182e-12,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.000434,0.000064,0.000002,0.000001,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.969601e-12,4.969601e-12,0.0,8.910363e-14,8.910363e-14,0.0,0.0,0.0
0,0.000924,0.000137,0.000004,0.000002,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,9.965421e-12,9.965421e-12,0.0,4.685310e-13,4.685310e-13,0.0,0.0,0.0
0,0.000705,0.000104,0.000003,0.000002,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.349686e-11,1.349686e-11,0.0,1.113339e-12,1.113339e-12,0.0,0.0,0.0
0,0.000769,0.000114,0.000002,0.000002,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.255710e-12,5.255710e-12,0.0,3.495766e-13,3.495766e-13,0.0,0.0,0.0


In [ ]:
ns_accumulated_df_valence = pd.DataFrame()
ns_accumulated_df_conduction = pd.DataFrame()
for s_select in range(nS):
    accumulated_df_valence = pd.DataFrame()
    accumulated_df_conduction = pd.DataFrame()
    for k_select in range(n_k):
        current_weighted_sum_valence = reversed_valence_df.loc[k_select+1].mul(contrib_v_dict[(s_select, k_select)], axis=0).sum()
        current_weighted_sum_conduction = conduction_df.loc[k_select+1].mul(contrib_c_dict[(s_select, k_select)], axis=0).sum()
        if accumulated_df_valence.empty:
            accumulated_df_valence = pd.DataFrame([current_weighted_sum_valence])
        else:
            accumulated_df_valence += current_weighted_sum_valence
        if accumulated_df_conduction.empty:
            accumulated_df_conduction = pd.DataFrame([current_weighted_sum_conduction])
        else:
            accumulated_df_conduction += current_weighted_sum_conduction
    ns_accumulated_df_valence = pd.concat([ns_accumulated_df_valence, accumulated_df_valence], axis=0)
    ns_accumulated_df_conduction = pd.concat([ns_accumulated_df_conduction, accumulated_df_conduction], axis=0)

print(accumulated_df_conduction)
#result_df
ns_accumulated_df_conduction


         0         1         2         3    4    5    6    7    8    9    ...  \
0  64.716237  9.459972  1.215197  0.952472  0.0  0.0  0.0  0.0  0.0  0.0  ...   

            650           651           652           653  654           655  \
0  1.410457e-10  1.414528e-10  6.729510e-08  6.729510e-08  0.0  1.035891e-09   

            656  657           658           659  
0  1.035891e-09  0.0  1.437415e-10  1.415609e-10  

[1 rows x 660 columns]


,0,1,2,3,4,5,6,7,8,9,...,650,651,652,653,654,655,656,657,658,659
0,85.407474,12.445419,1.295457,0.949981,0.0,0.0,0.0,0.0,0.0,0.0,...,3.038851e-09,3.039978e-09,9.784190e-07,9.784190e-07,0.0,1.401536e-08,1.401536e-08,0.0,3.154387e-09,3.046738e-09
0,84.126358,12.252681,1.295226,0.949915,0.0,0.0,0.0,0.0,0.0,0.0,...,3.075902e-09,3.076288e-09,1.001067e-06,1.001067e-06,0.0,1.440884e-08,1.440884e-08,0.0,3.194724e-09,3.084322e-09
0,71.322866,10.288797,1.273891,0.949612,0.0,0.0,0.0,0.0,0.0,0.0,...,1.108704e-09,1.678386e-09,8.207279e-07,8.207279e-07,0.0,8.240920e-09,8.240920e-09,0.0,2.060710e-09,1.661255e-09
0,70.367971,10.145363,1.274090,0.949572,0.0,0.0,0.0,0.0,0.0,0.0,...,1.160686e-09,1.734076e-09,8.379206e-07,8.379206e-07,0.0,8.521631e-09,8.521631e-09,0.0,2.117823e-09,1.716129e-09
0,66.800205,9.700959,1.220892,0.952336,0.0,0.0,0.0,0.0,0.0,0.0,...,1.743078e-10,1.746222e-10,5.660236e-08,5.660236e-08,0.0,1.008282e-09,1.008282e-09,0.0,1.769973e-10,1.747352e-10
0,53.654091,7.657538,1.187910,0.951705,0.0,0.0,0.0,0.0,0.0,0.0,...,1.953202e-11,4.670540e-11,1.552989e-08,1.552989e-08,0.0,2.965776e-10,2.965776e-10,0.0,6.610143e-11,4.637614e-11
0,72.869670,10.645800,1.214735,0.951478,0.0,0.0,0.0,0.0,0.0,0.0,...,3.033664e-10,3.018426e-10,8.811470e-08,8.811470e-08,0.0,1.671897e-09,1.671897e-09,0.0,3.035710e-10,3.039287e-10
0,62.089085,8.987761,1.214029,0.952165,0.0,0.0,0.0,0.0,0.0,0.0,...,1.347007e-10,1.352127e-10,4.979244e-08,4.979244e-08,0.0,7.763301e-10,7.763301e-10,0.0,1.374121e-10,1.350560e-10
0,52.135559,7.427939,1.187653,0.951666,0.0,0.0,0.0,0.0,0.0,0.0,...,1.699024e-11,4.012241e-11,1.810692e-08,1.810692e-08,0.0,2.741203e-10,2.741203e-10,0.0,5.636183e-11,3.991532e-11
0,64.716237,9.459972,1.215197,0.952472,0.0,0.0,0.0,0.0,0.0,0.0,...,1.410457e-10,1.414528e-10,6.729510e-08,6.729510e-08,0.0,1.035891e-09,1.035891e-09,0.0,1.437415e-10,1.415609e-10


In [ ]:
ns_accumulated_df_valence

,0,1,2,3,4,5,6,7,8,9,...,650,651,652,653,654,655,656,657,658,659
0,85.407474,12.445419,-0.146634,0.915961,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,84.126358,12.252681,-0.148506,0.917264,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,71.322866,10.288797,-0.117926,0.917960,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,70.367971,10.145363,-0.120236,0.919179,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,66.800205,9.700959,-0.106073,0.917668,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,53.654091,7.657538,-0.073194,0.919631,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,72.869670,10.645800,-0.109887,0.917072,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,62.089085,8.987761,-0.100928,0.920526,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,52.135559,7.427939,-0.074842,0.923298,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,64.716237,9.459972,-0.105060,0.918252,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
max(accumulated_df_conduction.iloc[:,4:].sum(axis=0))

0.1762402575616214

In [17]:
# Save comments and ns_accumulated_df_valence to a file, with every row in df is a single line
with open('pdos_sum_valence.dat', 'w') as f:
    for line in comments:
        f.write(line)
    for i in range(len(ns_accumulated_df_valence)):
        f.write(str(ns_accumulated_df_valence.iloc[i,:].values.tolist()).strip('[]') + '\n')


with open('pdos_sum_conduction.dat', 'w') as f:
    for line in comments:
        f.write(line)
    for i in range(len(ns_accumulated_df_conduction)):
        f.write(str(ns_accumulated_df_conduction.iloc[i,:].values.tolist()).strip('[]') + '\n')

## Step 4: Calculate the projectiong into different part of the heterostructure

This part is system specific, for now I just hard-coded it

### 4.1 for APD2_Pb

In [73]:
# Sum over the columns fro 37 to 72, 105 to 168 as the Br-per. contribution
# Sum over the columns fro 1 to 36, 73 to 104 as the Br-non. contribution
# Sum over the columns fro 169 to 232 as the O contribution
# Sum over the columns fro 609 to 624 as the Li contribution
# Total is 1 to 656

conduction_perovskite = ns_accumulated_df_conduction.iloc[:, 37+3:73+3].sum(axis=1) + ns_accumulated_df_conduction.iloc[:, 105+3:169+3].sum(axis=1)
conduction_non_perovskite = ns_accumulated_df_conduction.iloc[:, 1+3:37+3].sum(axis=1) + ns_accumulated_df_conduction.iloc[:, 73+3:105+3].sum(axis=1)
conduction_O = ns_accumulated_df_conduction.iloc[:, 169+3:233+3].sum(axis=1)
conduction_Li = ns_accumulated_df_conduction.iloc[:, 609+3:625+3].sum(axis=1)
conduction_total = ns_accumulated_df_conduction.iloc[:, 4:660].sum(axis=1)
conduction_other = ns_accumulated_df_conduction.iloc[:, 233+3:609+3].sum(axis=1) + ns_accumulated_df_conduction.iloc[:, 625+3:657+3].sum(axis=1)
conduction_others = conduction_total - conduction_perovskite - conduction_non_perovskite - conduction_O - conduction_Li - conduction_other

In [87]:
conduction_perovskite/ conduction_total

0    0.999791
0    0.999784
0    0.998604
0    0.998599
0    0.999986
0    0.999940
0    0.999973
0    0.999989
0    0.999940
0    0.999988
dtype: float64

In [81]:
# Do the same thing for the valence band
valence_perovskite = ns_accumulated_df_valence.iloc[:, 37+3:73+3].sum(axis=1) + ns_accumulated_df_valence.iloc[:, 105+3:169+3].sum(axis=1)
valence_non_perovskite = ns_accumulated_df_valence.iloc[:, 1+3:37+3].sum(axis=1) + ns_accumulated_df_valence.iloc[:, 73+3:105+3].sum(axis=1)
valence_O = ns_accumulated_df_valence.iloc[:, 169+3:233+3].sum(axis=1)
valence_Li = ns_accumulated_df_valence.iloc[:, 609+3:625+3].sum(axis=1)
valence_total = ns_accumulated_df_valence.iloc[:, 5:660].sum(axis=1)
valence_other = ns_accumulated_df_valence.iloc[:, 233+3:609+3].sum(axis=1) + ns_accumulated_df_valence.iloc[:, 625+3:657+3].sum(axis=1)
valence_others = valence_total - valence_perovskite - valence_non_perovskite - valence_O - valence_Li - valence_other

In [88]:
valence_non_perovskite/valence_total

0    0.327523
0    0.324113
0    0.358570
0    0.354084
0    0.368717
0    0.405801
0    0.368559
0    0.373969
0    0.401453
0    0.369129
dtype: float64

In [89]:
valence_O/valence_total

0    0.435511
0    0.437962
0    0.419789
0    0.422308
0    0.414135
0    0.394750
0    0.413273
0    0.410231
0    0.397657
0    0.414371
dtype: float64

In [90]:
valence_other/valence_total

0    0.236625
0    0.237571
0    0.221413
0    0.223363
0    0.217059
0    0.199402
0    0.218097
0    0.215706
0    0.200838
0    0.216457
dtype: float64

### 4.2 for Hypothetical COCN - Pb-Pb Intergrowth